# Prototypes of real-time analysis system for neural data

_Rotation student_: Conor McGrory
_Principal investigator_: Il Memming Park

## Background: Real-time analysis of neural data

The broad research objective that motivates this project is the development of a real-time analysis system for electrophysiological data. Being able to process neural data in real-time during the course of an experiment would enable us to perform a whole new class of experimental manipulations, including closed-loop experiments and custom perturbations of neural circuits.

![filter setup](img/filter_setup.png)

Our proposed setup uses two computers. The first, called the "probe" machine, interfaces directly with the probe used to collect the data (in our case, this will most likely be the [Neuropixel](https://www.neuropixels.org/)). At each time step, it reads data from the probe and sends it to the second computer, called the "processor," which uses the incoming data to update an adaptive filter or some other type of online data analysis method. In the case of a closed-loop experiment, the "processor" machine will send data back to the "probe" machine that will change some element of the experiment in response to a particular state of the filter.

One of the most significant technical challenges that needs to be overcome in order to make this system possible is the problem of _latency_. Modern probes, such as the Neuropixel, record data at very high sample rates, which in theory allows us to perform closed-loop experiments with a high level of temporal precision. However, the limiting factor currently preventing this is the time it takes for the analysis method to run. The broad aim of this project was to build simple prototypes of the real-time analysis system on a number of different platforms in order to understand how to minimize latency.

## Objectives

The main objectives for this project were:
1. Build simple prototypes of the real-time analysis system using three different languages: Python, Julia, and Rust
2. Measure the latency properties of all three prototypes and compare them


## Prototype setup

![prototype](./img/prototype.png)

The basic structure of the prototype is set up to mimic the conditions of an actual experiment as well as we can _in silico_. The probe and processor machines are simulated using two processes, which can either be run on two separate machines connected via Ethernet, or on a single machine using the local link hardware. In this setup, the processor machine functions as a server process, and the probe as a client. The processor, when run, waits for the probe to connect using a TCP/IP socket on a prespecified port. Once the probe is connected, it starts sending the test data, which consists of vectors of spike counts, across the socket, to the processor, which returns a "filter prediction" for each data point. The test data (see below) is loaded from an HDF5 file before the data transmission begins, in order to ensure that all measured latency is the result of the prototype itself.

When transmitting the data, the probe sends the spike count vectors one at a time, waiting until it recieves a response from the processor before it sends another one. While this "blocking" approach would clearly be suboptimal if used in a real experiment, it allows us to measure the round-trip latency -- the elapsed time between sending a spike vector and recieving its filter prediction -- in a straightforward and accurate way. In future prototypes, this approach will be replaced by an approach using two nonblocking sockets: one for sending spike data from probe to processor, and the other for feedback data from processor to probe.

### Test data

![raw data](./img/raw_data.png)

The neural data we use to test our prototypes is a publicly available dataset from [Steinmetz et al., 2019](https://www.nature.com/articles/s41586-019-1787-x). This data contains 698 neurons, and uses a bin size of 10 ms. Individual trials are 2.5 sec long, and contain 250 data points. The notebook [02_raw_data.ipynb](./02_raw_data.ipynb) contains the code used to download this data from the server and examine it locally.

Because the trials are relatively short, we concatenate all of the trials for a particular session into a longer signal, called the "test signal", for use with the prototypes. This is explained further in [03_test_signal.ipynb](./03_test_signal.ipynb), which contains the code used to create this signal. 

### Least-mean-squares filter

For the processor component of the prototypes, we used a simple adaptive filter called a [least-mean-squares (LMS) filter](https://en.wikipedia.org/wiki/Least_mean_squares_filter). Because this filter is linear, it isn't likely to be useful for any actual neural data analysis, but its ease of implementation and relatively low computational cost made it a good algorithm to use for measuring latency in different prototypes. For an exploration of LMS filtering and its application to our test data, see [04_lms_filter.ipynb](./04_lms_filter.ipynb).

![lms](./img/lms_synthetic.png)

### More complex filters

The initial goal of this project was to implement a more complex filter, like a Kalman filter, in the prototypes, to see what the latency of our system would be when using the kind of filter we would actually need to extract useful information from neural data. However, because we ended up spending more time than we expected decreasing the latency of the prototypes using the LMS filter, this will need to be left for future work.

## Results

## Project structure

- [02_raw_data.ipynb](./02_raw_data.ipynb)
- [03_test_signal.ipynb](./03_test_signal.ipynb)
- [04_lms_filter.ipynb](./04_lms_filter.ipynb)
- [05_results_python.ipynb](./05_results_python.ipynb)